In [6]:
import GetOldTweets3 as got
import pandas as pd

In [7]:
def get_tweets(query, top_only, start_date, end_date, max_tweets):
   
    # specifying tweet search criteria
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query)\
                                                .setTopTweets(top_only)\
                                                .setSince(start_date)\
                                                .setUntil(end_date)\
                                                .setMaxTweets(max_tweets)
    
    # scraping tweets based on criteria
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    
    # creating list of tweets with the tweet attributes
    # specified in the list comprehension
    text_tweets = [[tw.text,
                    tw.date,
                    tw.retweets,
                    tw.favorites,
                    tw.mentions,
                    tw.hashtags] for tw in tweet]
    
    # creating dataframe, assigning column names to list of
    # tweets corresponding to tweet attributes
    tweets_df = pd.DataFrame(text_tweets, 
                            columns = ['Text','Date','Retweets','Favorites','Mentions','HashTags'])
    
    return tweets_df

In [12]:
# Only including top tweets,
# Between a specific range of dates, with end date not inclusive
# And specifying that we want a max number of tweets in max_tweets.

# The main query word goes here, case insensitive
query = "lockdown"

tweets_df = get_tweets(query,
                     top_only = True,
                     start_date = "2020-08-01",
                     end_date = "2020-08-10",
                    max_tweets = 20)

tweets_df.head()

,Text,Date,Retweets,Favorites,Mentions,HashTags
0,Is it landlord that has been struggling since ...,2020-08-09 23:55:14+00:00,13,46,,
1,Victorian new cases trending down as impact of...,2020-08-09 23:53:42+00:00,8,29,,#auspol
2,Hello Monday. As Melbourne starts week two in ...,2020-08-09 23:48:01+00:00,7,23,,
3,The ironic part is the same people patting you...,2020-08-09 23:47:48+00:00,5,76,,
4,"In several states, positive coronavirus tests ...",2020-08-09 23:46:39+00:00,13,16,,


In [13]:
tweets_df.to_csv("tweets_df.csv")